# Multilayer Perceptron

In [1]:
from __future__ import division
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number
import numpy as np
# Enable inline plotting
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from scipy.linalg import svd
from sklearn.decomposition import TruncatedSVD

from sklearn.manifold import TSNE

from sklearn import preprocessing

from urllib import urlopen
from bs4 import BeautifulSoup
import nltk 

from dateutil import parser


from difflib import SequenceMatcher

import seaborn as sns
sns.set(style="darkgrid", palette="muted")
import matplotlib
matplotlib.style.use('ggplot')
pd.set_option('display.notebook_repr_html', True)
plt.rcParams['figure.figsize'] = 10, 6

/Users/valentin/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1035: UserWarning: Duplicate key in file "/Users/valentin/anaconda/lib/python2.7/site-packages/matplotlib/mpl-data/matplotlibrc", line #513
  (fname, cnt))


In [12]:
df_train = pd.read_csv('preprocessing_train_3.csv')
df_test = pd.read_csv('preprocessing_test_3.csv')
destinies = list(set(df_train['OutcomeType']))
labelEncoderDestinies = preprocessing.LabelEncoder()
labelEncoderDestinies.fit(destinies)
#print list(df_train.columns.values)

LabelEncoder()

In [4]:
preprocesseddf = df_train

from sklearn import cross_validation
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier 
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score,log_loss


features = list(set(df_test.columns.values).intersection(df_train.columns.values))

y = preprocesseddf["OutcomeType"]
X = preprocesseddf[features]

In [5]:
from sklearn.feature_extraction import DictVectorizer as DV
XX = preprocesseddf[features].values
yy = [ {'OutcomeType' : str(t)} for t in preprocesseddf['OutcomeType'].values]
vectorizer = DV( sparse = False )
yy = vectorizer.fit_transform(yy)

print vectorizer.get_feature_names()
#vec_x_cat_test = vectorizer.transform( x_cat_test )
# normalization
#XX = XX / XX.max(axis=0)
#yy = yy / yy.max(axis=0)
print yy
print XX

['OutcomeType=Adoption', 'OutcomeType=Died', 'OutcomeType=Euthanasia', 'OutcomeType=Return_to_owner', 'OutcomeType=Transfer']
[[ 0.  0.  0.  1.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 ..., 
 [ 1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]]
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [6]:
import tensorflow as tf
# define placeholderfor for images

Y_size = len(set(df_train.OutcomeType))
print "Y size: " + str(Y_size)
X_size = len(features)
print "X size: " + str(X_size)

Y size: 5
X size: 313


In [7]:
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

In [8]:
# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 128 # 1st layer number of features
n_hidden_2 = 128 # 2nd layer number of features
n_input = XX.shape[1] # entry size (20)
n_classes = yy.shape[1] # output size

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [9]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [10]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

In [11]:
def next_batch( X ,y ,i, N):
    return X[i : i+N, :], y[i:i+N, :]

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(XX.shape[0]/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = next_batch(XX,yy,i,batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost)
    print "Optimization Finished!"

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({x: XX, y: yy})

Epoch: 0001 cost= 4431.486581067
Epoch: 0002 cost= 270.156248028
Epoch: 0003 cost= 179.788091167
Epoch: 0004 cost= 206.996633337
Epoch: 0005 cost= 128.881551564
Epoch: 0006 cost= 99.414705962
Epoch: 0007 cost= 99.264367443
Epoch: 0008 cost= 99.982587093
Epoch: 0009 cost= 89.642976661
Epoch: 0010 cost= 75.495491456
Epoch: 0011 cost= 63.792800908
Epoch: 0012 cost= 73.805791887
Epoch: 0013 cost= 78.931978202
Epoch: 0014 cost= 50.125283748
Epoch: 0015 cost= 67.316293484
Optimization Finished!
Accuracy: 0.366381
